# Recommender Systems Deep-Dive Lab

There are many different approaches that we can take when creating recommender systems. In the Intro to Recommender Systems lesson and lab, we put together a user similarity based recommender that first calculated the similarities between users and then leveraged a rank-based item recommender within each group of similar customers. In other words, for a given user, our recommender found the top 5 customers who were the most similar to them, aggregated and ranked the purchases of those 5 customers, and then recommended the top 5 most popular products among that group of similar users to the customer.

In this lab, we are going to start out with the same data set, but we are going to dive deeper into the analysis of customers and products and look at an alternative way to generate recommendations.

We will begin by importing everything we will need for this lab (libraries, data set, etc.).

In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
#%timeit ,  line_profiler, memory_profiler

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

## Data Preparation

We will then put together the foundational transformations of the data that we will need to eventually produce recommendations. The steps in this section should be familiar to you, as you would have had to tranform the data in this manner to create the user similarity based recommender in the Intro to Recommender Systems lab.

First, we will create a data frame that contains the total quantity of each product purchased by each customer.

In [3]:
customer_products = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'}).reset_index()
customer_products.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


Then, we want to create a matrix that has customers on one axis, products on the other, and the quantity purchased as the values. There will be many instances where a customer has not purchased a product, which by default will be expressed with a null value. We will want to replace those nulls with zeros by appending `.fillna(0)` to our pivot table.

In [4]:
prod_cust_pivot = customer_products.pivot_table(values='Quantity', 
                                                columns='CustomerID', 
                                                index='ProductName', 
                                                aggfunc='sum').fillna(0)

prod_cust_pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,694,756,883,891,1008,1034,1066,1072,1336,1428,1435,1534,1577,1594,1754,1839,1920,2187,2329,2503,2556,2566,2582,2617,2686,2754,2776,2902,2915,2939,3074,3253,3267,3305,3317,3472,3531,3535,3544,3885,3903,3909,4261,4479,4595,4644,4723,5104,5177,5183,5224,5230,5434,5678,5739,5832,5921,5968,5986,5997,6001,6185,6317,6352,6367,6447,6543,6734,6817,6851,7021,7134,7223,7543,7606,7623,7772,7827,7987,7999,8316,8367,8372,8539,8711,8917,8962,9047,9132,9260,9302,9533,9546,9597,9771,9804,10016,10094,10114,10224,10311,10418,10739,11023,11057,11083,11102,11253,11325,11380,11441,11443,11614,11628,11744,11883,11933,12059,12206,12232,12381,12529,12561,12596,12598,12852,12888,13062,13118,13338,13354,13681,13699,13823,13912,14012,14208,14501,14782,14910,14913,14977,15027,15088,15119,15148,15165,15166,15371,15373,15415,15420,15672,15796,15851,15888,15920,15986,16139,16375,16481,16614,16650,16664,16809,16971,16997,17097,17136,17263,17390,17404,17407,17458,17472,17760,17765,17906,18027,18040,18133,18281,18293,18364,18441,18480,18552,18591,18622,18680,18683,18688,18706,18720,18728,18760,18796,19074,19147,19148,19260,19312,19442,19526,19753,20006,20125,20134,20331,20433,20446,20476,20648,20901,20907,21027,21095,21177,21734,21795,21973,22175,22204,22308,22322,22345,22408,22454,22461,22508,22536,22705,22751,22901,22939,22982,23001,23165,23176,23220,23377,23451,23515,23548,24137,24271,24323,24435,24490,24554,24820,25013,25040,25125,25209,25270,25292,25327,25420,25421,25449,25624,25651,25703,25779,25994,25995,26031,26353,26375,26458,26601,26850,26974,27045,27160,27209,27381,27410,27546,27572,27669,27672,27760,27821,27826,27841,27849,27857,28030,28191,28368,28702,28751,28756,28867,28875,28976,28984,29095,29258,29287,29342,29444,29475,29580,29656,29681,29721,29867,29910,29998,30196,30324,30423,30451,30575,30803,30815,30954,31075,31100,31113,31323,31373,31409,31468,31549,31820,32010,32028,32180,32324,32373,32386,32558,32575,32584,32682,32753,32785,32946,33051,33132,33320,33325,33381,33749,33759,33840,33878,34184,34291,34657,35028,35034,35107,35531,35575,35873,35877,36042,36138,36179,36200,36201,36566,36737,36740,36751,36829,37495,37510,37619,37873,38048,38125,38154,38186,38265,38307,38339,38358,38405,38497,38703,38798,38890,39110,39280,39537,39552,39757,39813,40094,40099,40223,40863,40896,40901,40934,40985,41045,41147,41286,41319,41405,41538,41722,41747,41829,41834,41843,42087,42157,42303,42306,42394,42399,42426,42459,42611,42612,42776,42802,42819,43270,43345,43529,43589,43652,44030,44058,44090,44349,44408,44667,44742,44859,45042,45266,45313,45420,45498,45549,45582,45598,45754,45759,45786,45846,45965,46036,46046,46146,46250,46315,46324,46367,46482,46551,46659,46715,46807,46826,46910,46971,47062,47122,47303,47305,47497,47605,47845,47913,47985,48108,48139,48156,48534,48611,48641,48642,48655,48818,48858,48963,49005,49159,49454,49474,49491,49795,49981,50004,50098,50416,50435,50461,50497,50535,50670,50748,50761,50774,51023,51121,51160,51182,51224,51279,51451,51483,51631,51718,51742,51834,51934,51939,52024,52091,52139,52154,52155,52479,52634,52700,53176,53198,53214,53226,53501,53729,53851,53997,54001,54093,54495,54547,54774,54827,54857,54894,54906,55124,55162,55245,55275,55281,55380,55381,55580,55715,55780,55875,55876,55930,55932,56319,56374,56382,56469,56479,56567,56786,56887,57024,57090,57168,57241,57317,57459,57497,57606,57654,57737,57977,57991,58064,58165,58203,58246,58321,58348,58353,58366,58413,58421,58535,58639,58652,58874,59017,59108,59150,59207,59258,59379,59399,59626,59703,59710,59711,59831,59925,60185,60378,60389,60481,60515,60612,60759,60781,60862,60917,60938,60969,61223,61288,61417,61601,61750,61923,62005,62034,62094,62101,62184,62242,62277,62279,62411,62421,62592,62719,63033,63086,63120,63295,63368,63742,63771,63803,63891,63963,64117,64202,64301,64348,64575,64597,64615,64775,64787,64862,65326,65535,65650,65678,65731,65757,65785,65830,65875,65910,65925,66177,66232,66320,66419,66445,66632,66709,66807,66822,

In the pivot table we created, the rows represent the products and the columns represent the customers. Depending on what need to do with the matrix, we may instead need to transpose it so that the rows represent customers and the columns to represent products. We can do this easily by appending `.T` to our product customer matrix.

In [5]:
cust_prod_pivot = prod_cust_pivot.T
cust_prod_pivot.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,Bacardi Breezer - Tropical,Bagel - Plain,Baking Powder,Banana - Leaves,Banana Turning,Bananas,Bandage - Fexible 1x3,Bandage - Flexible Neon,Bar - Granola Trail Mix Fruit Nut,"Bar Mix - Pina Colada, 355 Ml",Barramundi,Bay Leaf,Beans - Kidney White,"Beans - Kidney, Canned","Beans - Kidney, Red Dry",Beans - Wax,"Beef - Chuck, Boneless",Beef - Ground Medium,"Beef - Ground, Extra Lean, Fresh",Beef - Inside Round,Beef - Montreal Smoked Brisket,Beef - Prime Rib Aaa,Beef - Rib Eye Aaa,Beef - Short Loin,Beef - Striploin Aa,"Beef - Tenderlion, Center Cut",Beef - Texas Style Burger,Beef - Top Sirloin,Beef - Top Sirloin - Aaa,Beef Ground Medium,Beef Wellington,"Beer - Alexander Kieths, Pale Ale",Beer - Blue,Beer - Labatt Blue,Beer - Original Organic Lager,Beer - Rickards Red,Beer - Sleemans Cream Ale,"Beets - Candy Cane, Organic",Beets - Mini Golden,Berry Brulee,Black Currants,Blackberries,Blueberries,Bouq All Italian - Primerba,Brandy - Bar,Bread - Bistro White,Bread - Calabrese Baguette,Bread - English Muffin,Bread - French Baquette,Bread - Hot Dog Buns,Bread - Italian Corn Meal Poly,Bread - Italian Roll With Herbs,Bread - Multigrain,Bread - Raisin Walnut Oval,"Bread - Roll, Canadian Dinner","Bread - Roll, Soft White Round",Bread - Rye,Bread Crumbs - Japanese Style,Bread Crumbs - Panko,Bread Fig And Almond,Bread Foccacia Whole,Broom - Corn,Browning Caramel Glace,Butter - Unsalted,Cake - Box Window 10x10x2.5,Cake - Cake Sheet Macaroon,Cake - Mini Cheesecake,Campari,Carbonated Water - Blackcherry,Carbonated Water - Cherry,Cassis,Cattail Hearts,Cheese - Bocconcini,"Cheese - Boursin, Garlic / Herbs","Cheese - Brie, Triple Creme","Cheese - Brie,danish",Cheese - Cambozola,Cheese - Camembert,Cheese - Cheddarsliced,Cheese - Cottage Cheese,Cheese - Mix,Cheese - Mozzarella,Cheese - Parmesan Cubes,Cheese - Parmesan Grated,Cheese - Taleggio D.o.p.,Cheese - Victor Et Berthold,Cheese - Wine,Cheese Cloth No 100,Chef Hat 20cm,"Chestnuts - Whole,canned","Chicken - Leg, Boneless",Chicken - Soup Base,Chicken - Wieners,Chinese Foods - Chicken,Chips Potato All Dressed - 43g,Chips Potato Salt Vinegar 43g,Chocolate - Compound Coating,Chocolate - Dark,Chocolate - Feathers,"Chocolate - Semi Sweet, Calets",Cinnamon Buns Sticky,Clam Nectar,Cocktail Napkin Blue,Cocoa Butter,"Coconut - Shredded, Sweet",Cod - Black Whole Fillet,Coffee - Dark Roast,Coffee - Hazelnut Cream,Coffee - Irish Cream,Coffee Decaf Colombian,Cookie - Dough Variety,Cookie Chocolate Chip With,Cookie Dough - Double,Cookies - Assorted,Cookies Cereal Nut,Corn Meal,Cornflakes,"Crab - Dungeness, Whole",Crab - Imitation Flakes,Crackers - Trio,Crackers Cheez It,Cream Of Tartar,Creme De Banane - Marie,Crush - Cream Soda,Cumin - Whole,"Cup - 6oz, Foam",Cup - Translucent 7 Oz Clear,Curry Paste - Madras,Dc - Frozen Momji,Dc Hikiage Hira Huba,"Doilies - 5, Paper",Dried Figs,Duck - Breast,Durian Fruit,Ecolab - Lime - A - Way 4/4 L,Ecolab - Mikroklene 4/4 L,Ecolab - Solid Fusion,Eggplant - Asian,Extract - Lemon,Ezy Change Mophandle,Fenngreek Seed,"Fish - Scallops, Cold Smoked",Flavouring - Orange,Flour - Pastry,Flour - Teff,Flour - Whole Wheat,Foam Cup 6 Oz,Foam Dinner Plate,Fond - Neutral,Fondant - Icing,French Pastry - Mini Chocolate,Fuji Apples,Garbag Bags - Black,Garbage Bags - Clear,Garlic,Garlic - Elephant,Garlic - Peeled,"Garlic - Primerba, Paste",Gatorade - Xfactor Berry,General Purpose Trigger,Gloves - Goldtouch Disposable,Grapes - Red,Grenadine,Grouper - Fresh,Guinea Fowl,Halibut - Fletches,Halibut - Steaks,Hersey Shakes,"Hickory Smoke, Liquid",Hinge W Undercut,Hot Chocolate - Individual,"Ice - Clear, 300 Lb For Carving",Ice Cream Bar - Drumstick,Ice Cream Bar - Hageen Daz To,Ice Cream Bar - Oreo Cone,Initation Crab Meat,Island Oasis - Mango Daiquiri,Isomalt,Jagermei

Another thing we may want to do is normalize the values across rows or columns of the matrix so that all the values are between 0 and 1. Doing this for customers would help us identify customers that may have purcahsed a similar mix of products even though some of those customers may have purchased large quantities while other may have purchased smaller quantities. Doing this for products would help us better identify products that have been purchased by similar groups of customers regardless of the quantities purchased.

We can normalize across rows for each matrix as follows.

In [6]:
prod_cust_pivot = prod_cust_pivot.div(prod_cust_pivot.sum(axis=1), axis=0)
cust_prod_pivot = cust_prod_pivot.div(cust_prod_pivot.sum(axis=1), axis=0)
prod_cust_pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,694,756,883,891,1008,1034,1066,1072,1336,1428,1435,1534,1577,1594,1754,1839,1920,2187,2329,2503,2556,2566,2582,2617,2686,2754,2776,2902,2915,2939,3074,3253,3267,3305,3317,3472,3531,3535,3544,3885,3903,3909,4261,4479,4595,4644,4723,5104,5177,5183,5224,5230,5434,5678,5739,5832,5921,5968,5986,5997,6001,6185,6317,6352,6367,6447,6543,6734,6817,6851,7021,7134,7223,7543,7606,7623,7772,7827,7987,7999,8316,8367,8372,8539,8711,8917,8962,9047,9132,9260,9302,9533,9546,9597,9771,9804,10016,10094,10114,10224,10311,10418,10739,11023,11057,11083,11102,11253,11325,11380,11441,11443,11614,11628,11744,11883,11933,12059,12206,12232,12381,12529,12561,12596,12598,12852,12888,13062,13118,13338,13354,13681,13699,13823,13912,14012,14208,14501,14782,14910,14913,14977,15027,15088,15119,15148,15165,15166,15371,15373,15415,15420,15672,15796,15851,15888,15920,15986,16139,16375,16481,16614,16650,16664,16809,16971,16997,17097,17136,17263,17390,17404,17407,17458,17472,17760,17765,17906,18027,18040,18133,18281,18293,18364,18441,18480,18552,18591,18622,18680,18683,18688,18706,18720,18728,18760,18796,19074,19147,19148,19260,19312,19442,19526,19753,20006,20125,20134,20331,20433,20446,20476,20648,20901,20907,21027,21095,21177,21734,21795,21973,22175,22204,22308,22322,22345,22408,22454,22461,22508,22536,22705,22751,22901,22939,22982,23001,23165,23176,23220,23377,23451,23515,23548,24137,24271,24323,24435,24490,24554,24820,25013,25040,25125,25209,25270,25292,25327,25420,25421,25449,25624,25651,25703,25779,25994,25995,26031,26353,26375,26458,26601,26850,26974,27045,27160,27209,27381,27410,27546,27572,27669,27672,27760,27821,27826,27841,27849,27857,28030,28191,28368,28702,28751,28756,28867,28875,28976,28984,29095,29258,29287,29342,29444,29475,29580,29656,29681,29721,29867,29910,29998,30196,30324,30423,30451,30575,30803,30815,30954,31075,31100,31113,31323,31373,31409,31468,31549,31820,32010,32028,32180,32324,32373,32386,32558,32575,32584,32682,32753,32785,32946,33051,33132,33320,33325,33381,33749,33759,33840,33878,34184,34291,34657,35028,35034,35107,35531,35575,35873,35877,36042,36138,36179,36200,36201,36566,36737,36740,36751,36829,37495,37510,37619,37873,38048,38125,38154,38186,38265,38307,38339,38358,38405,38497,38703,38798,38890,39110,39280,39537,39552,39757,39813,40094,40099,40223,40863,40896,40901,40934,40985,41045,41147,41286,41319,41405,41538,41722,41747,41829,41834,41843,42087,42157,42303,42306,42394,42399,42426,42459,42611,42612,42776,42802,42819,43270,43345,43529,43589,43652,44030,44058,44090,44349,44408,44667,44742,44859,45042,45266,45313,45420,45498,45549,45582,45598,45754,45759,45786,45846,45965,46036,46046,46146,46250,46315,46324,46367,46482,46551,46659,46715,46807,46826,46910,46971,47062,47122,47303,47305,47497,47605,47845,47913,47985,48108,48139,48156,48534,48611,48641,48642,48655,48818,48858,48963,49005,49159,49454,49474,49491,49795,49981,50004,50098,50416,50435,50461,50497,50535,50670,50748,50761,50774,51023,51121,51160,51182,51224,51279,51451,51483,51631,51718,51742,51834,51934,51939,52024,52091,52139,52154,52155,52479,52634,52700,53176,53198,53214,53226,53501,53729,53851,53997,54001,54093,54495,54547,54774,54827,54857,54894,54906,55124,55162,55245,55275,55281,55380,55381,55580,55715,55780,55875,55876,55930,55932,56319,56374,56382,56469,56479,56567,56786,56887,57024,57090,57168,57241,57317,57459,57497,57606,57654,57737,57977,57991,58064,58165,58203,58246,58321,58348,58353,58366,58413,58421,58535,58639,58652,58874,59017,59108,59150,59207,59258,59379,59399,59626,59703,59710,59711,59831,59925,60185,60378,60389,60481,60515,60612,60759,60781,60862,60917,60938,60969,61223,61288,61417,61601,61750,61923,62005,62034,62094,62101,62184,62242,62277,62279,62411,62421,62592,62719,63033,63086,63120,63295,63368,63742,63771,63803,63891,63963,64117,64202,64301,64348,64575,64597,64615,64775,64787,64862,65326,65535,65650,65678,65731,65757,65785,65830,65875,65910,65925,66177,66232,66320,66419,66445,66632,66709,66807,66822,

## User Similarity Based Recommendations

The next step in creating recommendations is calculating similarities. For our user similarity based recommender, we calculated them between customers.

In [7]:
cust_dist = pd.DataFrame(1/(1 + squareform(pdist(cust_prod_pivot, 'euclidean'))),
                         index=cust_prod_pivot.index, columns=cust_prod_pivot.index)

cust_dist.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,694,756,883,891,1008,1034,1066,1072,1336,1428,1435,1534,1577,1594,1754,1839,1920,2187,2329,2503,2556,2566,2582,2617,2686,2754,2776,2902,2915,2939,3074,3253,3267,3305,3317,3472,3531,3535,3544,3885,3903,3909,4261,4479,4595,4644,4723,5104,5177,5183,5224,5230,5434,5678,5739,5832,5921,5968,5986,5997,6001,6185,6317,6352,6367,6447,6543,6734,6817,6851,7021,7134,7223,7543,7606,7623,7772,7827,7987,7999,8316,8367,8372,8539,8711,8917,8962,9047,9132,9260,9302,9533,9546,9597,9771,9804,10016,10094,10114,10224,10311,10418,10739,11023,11057,11083,11102,11253,11325,11380,11441,11443,11614,11628,11744,11883,11933,12059,12206,12232,12381,12529,12561,12596,12598,12852,12888,13062,13118,13338,13354,13681,13699,13823,13912,14012,14208,14501,14782,14910,14913,14977,15027,15088,15119,15148,15165,15166,15371,15373,15415,15420,15672,15796,15851,15888,15920,15986,16139,16375,16481,16614,16650,16664,16809,16971,16997,17097,17136,17263,17390,17404,17407,17458,17472,17760,17765,17906,18027,18040,18133,18281,18293,18364,18441,18480,18552,18591,18622,18680,18683,18688,18706,18720,18728,18760,18796,19074,19147,19148,19260,19312,19442,19526,19753,20006,20125,20134,20331,20433,20446,20476,20648,20901,20907,21027,21095,21177,21734,21795,21973,22175,22204,22308,22322,22345,22408,22454,22461,22508,22536,22705,22751,22901,22939,22982,23001,23165,23176,23220,23377,23451,23515,23548,24137,24271,24323,24435,24490,24554,24820,25013,25040,25125,25209,25270,25292,25327,25420,25421,25449,25624,25651,25703,25779,25994,25995,26031,26353,26375,26458,26601,26850,26974,27045,27160,27209,27381,27410,27546,27572,27669,27672,27760,27821,27826,27841,27849,27857,28030,28191,28368,28702,28751,28756,28867,28875,28976,28984,29095,29258,29287,29342,29444,29475,29580,29656,29681,29721,29867,29910,29998,30196,30324,30423,30451,30575,30803,30815,30954,31075,31100,31113,31323,31373,31409,31468,31549,31820,32010,32028,32180,32324,32373,32386,32558,32575,32584,32682,32753,32785,32946,33051,33132,33320,33325,33381,33749,33759,33840,33878,34184,34291,34657,35028,35034,35107,35531,35575,35873,35877,36042,36138,36179,36200,36201,36566,36737,36740,36751,36829,37495,37510,37619,37873,38048,38125,38154,38186,38265,38307,38339,38358,38405,38497,38703,38798,38890,39110,39280,39537,39552,39757,39813,40094,40099,40223,40863,40896,40901,40934,40985,41045,41147,41286,41319,41405,41538,41722,41747,41829,41834,41843,42087,42157,42303,42306,42394,42399,42426,42459,42611,42612,42776,42802,42819,43270,43345,43529,43589,43652,44030,44058,44090,44349,44408,44667,44742,44859,45042,45266,45313,45420,45498,45549,45582,45598,45754,45759,45786,45846,45965,46036,46046,46146,46250,46315,46324,46367,46482,46551,46659,46715,46807,46826,46910,46971,47062,47122,47303,47305,47497,47605,47845,47913,47985,48108,48139,48156,48534,48611,48641,48642,48655,48818,48858,48963,49005,49159,49454,49474,49491,49795,49981,50004,50098,50416,50435,50461,50497,50535,50670,50748,50761,50774,51023,51121,51160,51182,51224,51279,51451,51483,51631,51718,51742,51834,51934,51939,52024,52091,52139,52154,52155,52479,52634,52700,53176,53198,53214,53226,53501,53729,53851,53997,54001,54093,54495,54547,54774,54827,54857,54894,54906,55124,55162,55245,55275,55281,55380,55381,55580,55715,55780,55875,55876,55930,55932,56319,56374,56382,56469,56479,56567,56786,56887,57024,57090,57168,57241,57317,57459,57497,57606,57654,57737,57977,57991,58064,58165,58203,58246,58321,58348,58353,58366,58413,58421,58535,58639,58652,58874,59017,59108,59150,59207,59258,59379,59399,59626,59703,59710,59711,59831,59925,60185,60378,60389,60481,60515,60612,60759,60781,60862,60917,60938,60969,61223,61288,61417,61601,61750,61923,62005,62034,62094,62101,62184,62242,62277,62279,62411,62421,62592,62719,63033,63086,63120,63295,63368,63742,63771,63803,63891,63963,64117,64202,64301,64348,64575,64597,64615,64775,64787,64862,65326,65535,65650,65678,65731,65757,65785,65830,65875,65910,65925,66177,66232,66320,66419,66445,66632,66709,66807,66822,

Once we had our similarity matrix, then we could produce recommendations for each user and package all the recommendations into a data frame.

In [8]:
recommendations = {}
customers = list(customer_products['CustomerID'].unique())

for customer in customers:
    similar_cust = list(cust_dist[customer].sort_values(ascending=False)[1:].head().index)
    sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
    grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
    ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
    
    merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[customer]), on='ProductName')
    merged.columns = ['ProductName', 'Quantity', 'Purchased']
    recs = merged[merged['Purchased']==0].head()
    recommendations[customer] = list(recs['ProductName'])

user_recs = pd.DataFrame.from_dict(recommendations, orient='index').reset_index()
user_recs.columns = ['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
user_recs.head()

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,33,Wine - Redchard Merritt,Bread - Calabrese Baguette,"Thyme - Lemon, Fresh",Milk Powder,Ecolab - Lime - A - Way 4/4 L
1,200,Sauce - Demi Glace,General Purpose Trigger,Cookie Chocolate Chip With,Chef Hat 20cm,Pasta - Angel Hair
2,264,Ezy Change Mophandle,Eggplant - Asian,Scallops - 10/20,Cinnamon Buns Sticky,Wine - Ej Gallo Sierra Valley
3,356,Tea - Herbal Sweet Dreams,Curry Paste - Madras,Tea - English Breakfast,Juice - Orange,Ecolab - Lime - A - Way 4/4 L
4,412,Cake - Box Window 10x10x2.5,Beef - Montreal Smoked Brisket,Bread - Raisin Walnut Oval,"Mushroom - Trumpet, Dry",Cheese - Mix


## Deeper Dive Into Our User Similarity Recommendations

Let's deconstruct what we've done and take a deeper dive into how we put this together. Doing this will equip us with the knowledge to be able to put together an item-based similarity recommender in the next section. 

After creating an empty dictionary to store our recommendations and getting a unique list of customer IDs to iterate through, we are first identifying the top 5 similar customers to the customer we are trying to generate recommendations for. Let's plug in customer ID 33 and see what results we get.

In [9]:
similar_cust = list(cust_dist[33].sort_values(ascending=False)[1:].head().index)
similar_cust

[60862, 27672, 6001, 79458, 33759]

What we get is a list containing the 5 customer IDs of the customers whose purchase behavior is most similar to customer 33. We then go back to our customer_products data frame and select just the purchases where the customer ID is in our list of similar customers. We aggregate on product name, summing up the total quantity purchased of each product by all 5 similar customers, and then we rank them by sorting in descending order by the total quantity.

In [10]:
sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
ranked_products.head()

,ProductName,Quantity
0,Wine - Redchard Merritt,59
1,Cassis,58
2,Bread - Calabrese Baguette,45
3,Wine - Crozes Hermitage E.,45
4,"Thyme - Lemon, Fresh",42


We now have a ranked list of products that similar customers have purchased, but we haven't taken into consideration yet whether our target customer already purchases any of those items. We want to recommend them items that they might like but haven't purchased before. So we will merge the list of ranked products with our target customer's purchase list and keep only the records for items that the customer has not purchased. These will be the items that we recommend to the customer.

In [11]:
merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[33]), on='ProductName')
merged.columns = ['ProductName', 'Quantity', 'Purchased']
recs = merged[merged['Purchased']==0].head()
recs

,ProductName,Quantity,Purchased
0,Wine - Redchard Merritt,59,0.0
2,Bread - Calabrese Baguette,45,0.0
4,"Thyme - Lemon, Fresh",42,0.0
9,Milk Powder,39,0.0
10,Ecolab - Lime - A - Way 4/4 L,39,0.0


## Item Similarity Based Recommendations

In this section, you will create an item similarity based recommender system in a step-by-step fashion. Whereas our user similarity based recommender leveraged similarities between customers, this recommender will utilize similarities between products. You already have all the tools in your toolbox, so follow each of the steps below to complete this lab.

### Step 1: Create a product distance matrix.

In [12]:
prod_dist = pd.DataFrame(1/(1 + squareform(pdist(prod_cust_pivot, 'euclidean'))),
                         index=prod_cust_pivot.index, columns=prod_cust_pivot.index)
prod_dist.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,Bacardi Breezer - Tropical,Bagel - Plain,Baking Powder,Banana - Leaves,Banana Turning,Bananas,Bandage - Fexible 1x3,Bandage - Flexible Neon,Bar - Granola Trail Mix Fruit Nut,"Bar Mix - Pina Colada, 355 Ml",Barramundi,Bay Leaf,Beans - Kidney White,"Beans - Kidney, Canned","Beans - Kidney, Red Dry",Beans - Wax,"Beef - Chuck, Boneless",Beef - Ground Medium,"Beef - Ground, Extra Lean, Fresh",Beef - Inside Round,Beef - Montreal Smoked Brisket,Beef - Prime Rib Aaa,Beef - Rib Eye Aaa,Beef - Short Loin,Beef - Striploin Aa,"Beef - Tenderlion, Center Cut",Beef - Texas Style Burger,Beef - Top Sirloin,Beef - Top Sirloin - Aaa,Beef Ground Medium,Beef Wellington,"Beer - Alexander Kieths, Pale Ale",Beer - Blue,Beer - Labatt Blue,Beer - Original Organic Lager,Beer - Rickards Red,Beer - Sleemans Cream Ale,"Beets - Candy Cane, Organic",Beets - Mini Golden,Berry Brulee,Black Currants,Blackberries,Blueberries,Bouq All Italian - Primerba,Brandy - Bar,Bread - Bistro White,Bread - Calabrese Baguette,Bread - English Muffin,Bread - French Baquette,Bread - Hot Dog Buns,Bread - Italian Corn Meal Poly,Bread - Italian Roll With Herbs,Bread - Multigrain,Bread - Raisin Walnut Oval,"Bread - Roll, Canadian Dinner","Bread - Roll, Soft White Round",Bread - Rye,Bread Crumbs - Japanese Style,Bread Crumbs - Panko,Bread Fig And Almond,Bread Foccacia Whole,Broom - Corn,Browning Caramel Glace,Butter - Unsalted,Cake - Box Window 10x10x2.5,Cake - Cake Sheet Macaroon,Cake - Mini Cheesecake,Campari,Carbonated Water - Blackcherry,Carbonated Water - Cherry,Cassis,Cattail Hearts,Cheese - Bocconcini,"Cheese - Boursin, Garlic / Herbs","Cheese - Brie, Triple Creme","Cheese - Brie,danish",Cheese - Cambozola,Cheese - Camembert,Cheese - Cheddarsliced,Cheese - Cottage Cheese,Cheese - Mix,Cheese - Mozzarella,Cheese - Parmesan Cubes,Cheese - Parmesan Grated,Cheese - Taleggio D.o.p.,Cheese - Victor Et Berthold,Cheese - Wine,Cheese Cloth No 100,Chef Hat 20cm,"Chestnuts - Whole,canned","Chicken - Leg, Boneless",Chicken - Soup Base,Chicken - Wieners,Chinese Foods - Chicken,Chips Potato All Dressed - 43g,Chips Potato Salt Vinegar 43g,Chocolate - Compound Coating,Chocolate - Dark,Chocolate - Feathers,"Chocolate - Semi Sweet, Calets",Cinnamon Buns Sticky,Clam Nectar,Cocktail Napkin Blue,Cocoa Butter,"Coconut - Shredded, Sweet",Cod - Black Whole Fillet,Coffee - Dark Roast,Coffee - Hazelnut Cream,Coffee - Irish Cream,Coffee Decaf Colombian,Cookie - Dough Variety,Cookie Chocolate Chip With,Cookie Dough - Double,Cookies - Assorted,Cookies Cereal Nut,Corn Meal,Cornflakes,"Crab - Dungeness, Whole",Crab - Imitation Flakes,Crackers - Trio,Crackers Cheez It,Cream Of Tartar,Creme De Banane - Marie,Crush - Cream Soda,Cumin - Whole,"Cup - 6oz, Foam",Cup - Translucent 7 Oz Clear,Curry Paste - Madras,Dc - Frozen Momji,Dc Hikiage Hira Huba,"Doilies - 5, Paper",Dried Figs,Duck - Breast,Durian Fruit,Ecolab - Lime - A - Way 4/4 L,Ecolab - Mikroklene 4/4 L,Ecolab - Solid Fusion,Eggplant - Asian,Extract - Lemon,Ezy Change Mophandle,Fenngreek Seed,"Fish - Scallops, Cold Smoked",Flavouring - Orange,Flour - Pastry,Flour - Teff,Flour - Whole Wheat,Foam Cup 6 Oz,Foam Dinner Plate,Fond - Neutral,Fondant - Icing,French Pastry - Mini Chocolate,Fuji Apples,Garbag Bags - Black,Garbage Bags - Clear,Garlic,Garlic - Elephant,Garlic - Peeled,"Garlic - Primerba, Paste",Gatorade - Xfactor Berry,General Purpose Trigger,Gloves - Goldtouch Disposable,Grapes - Red,Grenadine,Grouper - Fresh,Guinea Fowl,Halibut - Fletches,Halibut - Steaks,Hersey Shakes,"Hickory Smoke, Liquid",Hinge W Undercut,Hot Chocolate - Individual,"Ice - Clear, 300 Lb For Carving",Ice Cream Bar - Drumstick,Ice Cream Bar - Hageen Daz To,Ice Cream Bar - Oreo Cone,Initation Crab Meat,Island Oasis - Mango Daiquiri,Isomalt,Jagermei

### Step 2: Get the products purchased for a specific customer of your choice.

In [13]:
cust_33 = customer_products[customer_products['CustomerID']==33]
cust_33.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


### Step 3: For each product the customer purchased, get a list of the top 5 similar products. Package the lists into a nested list, flatten the list, and then filter out any products the customer has already purchased.

In [14]:
list_prods = list(cust_33.ProductName.unique())

similars = []
for prod in list_prods:
    similar_prod = list(prod_dist[prod].sort_values(ascending=False)[1:].head().index)
    similars.extend(similar_prod)
similars = list(set(similars))

closest = [pr for pr in similars if pr not in list_prods]

grouped_products = customer_products.groupby('ProductName',as_index=False).agg({'Quantity':'count'})
grouped_products.head()
prods_not_purchased = grouped_products[grouped_products['ProductName'].isin(closest)]


,ProductName,Quantity
0,Anchovy Paste - 56 G Tube,144
1,"Appetizer - Mini Egg Roll, Shrimp",151
2,Appetizer - Mushroom Tart,140
3,Appetizer - Sausage Rolls,125
4,Apricots - Dried,143


### Step 4: Count the number of times each similar product occurs in your filtered list. Sort and return a list containing the top 5 items.

In [15]:
prods_not_purchased.sort_values(by='Quantity',ascending=False)[:5]

,ProductName,Quantity
353,Spinach - Baby,172
123,Cookies - Assorted,168
247,Oil - Shortening - All - Purpose,166
391,V8 - Berry Blend,163
226,Muffin - Carrot Individual Wrap,163


### Step 5: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 2 through 4 for each and appending the results of each iteration to the dictionary you created.

In [16]:
from IPython.display import clear_output

def output(i, end=False):
    global prev, length
    res = round(i / length * 100, 2)
    if res - prev >= 0.5 or end:
        clear_output()
        if end: res = 100.0
        display('Completed: '+str(res)+ '%')
        prev = res
        
def get_unique_prods(data):
    return list(data.ProductName.unique())

def get_recommendations(cust_id, list_prods=None):
    cust_ = customer_products[customer_products['CustomerID']==cust_id] 
    if not list_prods:
        list_prods = list(cust_.ProductName.unique())
    similars = []
    for prod in list_prods:
        similar_prod = list(prod_dist[prod].sort_values(ascending=False)[1:].head().index)
        similars.extend(similar_prod)
    similars = list(set(similars))
    closest = [pr for pr in similars if pr not in list_prods]
    grouped_products = customer_products.groupby('ProductName',as_index=False).agg({'Quantity':'count'})
    prods_not_purchased = grouped_products[grouped_products['ProductName'].isin(closest)]
    total = prods_not_purchased.sort_values(by='Quantity',ascending=False)[:5]
    return list(total['ProductName'])
    

In [17]:
recommendations = {}

customers = list(customer_products['CustomerID'].unique())

global prev, length
prev, length = 0, len(customers)
for i,customer in enumerate(customers):
    output(i)
    recommendations[customer] = get_recommendations(customer)
output(i, end=True)

'Completed: 100.0%'

### Step 6: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [18]:
prods = pd.DataFrame.from_dict(recommendations, orient='index').reset_index()
prods.columns = ['CustomerID', 'P1', 'P2', 'P3', 'P4', 'P5']
prods.head()

,CustomerID,P1,P2,P3,P4,P5
0,33,Spinach - Baby,Cookies - Assorted,Oil - Shortening - All - Purpose,V8 - Berry Blend,Muffin - Carrot Individual Wrap
1,200,Spinach - Baby,Cookies - Assorted,Oil - Shortening - All - Purpose,Hersey Shakes,Puree - Passion Fruit
2,264,Spinach - Baby,Cookies - Assorted,Oil - Shortening - All - Purpose,"Sole - Dover, Whole, Fresh",Puree - Passion Fruit
3,356,Spinach - Baby,Cookies - Assorted,Oil - Shortening - All - Purpose,"Sole - Dover, Whole, Fresh",Puree - Passion Fruit
4,412,Oil - Shortening - All - Purpose,"Sole - Dover, Whole, Fresh",Scallops - Live In Shell,Puree - Passion Fruit,V8 - Berry Blend


## Recommending Items to a New Customer

Suppose we get a new customer and on their first visit, they purchase the following items and quantities.

In [19]:
new_customer = {'Cookies - Assorted':3,
                'Flavouring - Orange':3,
                'Fenngreek Seed':1,
                'Wine - White Cab Sauv.on':1,
                'Bandage - Flexible Neon':3,
                'Oil - Shortening - All - Purpose':2,
                'Beef - Montreal Smoked Brisket':4,
                'French Pastry - Mini Chocolate':4,
                'Snapple Lemon Tea':5,
                'Pepper - White, Ground':2,
                'Spinach - Baby':5,
                'Sole - Dover, Whole, Fresh':4}

### Step 7: Recommend 5 products to this new customer using a user similarity approach.

In [20]:
def get_5_close(cus_id, distances):
    results_5 = distances[cus_id].sort_values(ascending=False)[1:6]
    return list(results_5.keys())

def get_ranked_by5(cus_id, distances, buy_each):
    close_5 = get_5_close(cus_id, distances)
    similar = buy_each.loc[buy_each.CustomerID.isin(close_5)]
    ranked = similar.groupby(['ProductName'],as_index=False).agg({'Quantity':'sum'}).sort_values('Quantity', ascending=False)
    ranked.reset_index(drop=True)
    return ranked
    
def get_ranked_not_buyed_by(customer_id, ranked, data):
    buy_by_customer = data[data['CustomerID']==customer_id]['ProductName']
    ranked_not = ranked.loc[~ranked['ProductName'].isin(customer)]
    return ranked_not

def get_filtered_matrix(cus_id, cus_matrix, ranked):
    return pd.merge(cus_matrix,get_ranked_not_buyed_by(cus_id, ranked, data), on='ProductName')[:5]

def get_dic(filtered):
        alist = filtered['ProductName'].values.tolist()
        di = {}
        for i, item in enumerate(alist):
            di['product'+str(i+1)] = item
        return di

def get_customers_ids(data):
    return data['CustomerID'].values.tolist()
    

In [21]:

prods_new_customer = pd.DataFrame.from_dict(new_customer, orient='index').reset_index()
prods_new_customer.columns = ['ProductName', 'Quantity']
prods_new_customer['CustomerID'] = '100001'
print('prods_new_customer')
prods_new_customer.head()

customer_products_search = pd.concat([prods_new_customer,customer_products], axis=0)

print('*****  RECOMMENDATIONS  *****')

distance_method = 'euclidean'
cus_id = '100001'
buy_each = customer_products_search.groupby(['CustomerID','ProductName'],as_index=False).agg({'Quantity':'sum'})
cus_matrix = buy_each.pivot_table(index='ProductName', columns='CustomerID', values='Quantity').fillna(0)
distances = pd.DataFrame(1/(1 + squareform(pdist(cus_matrix.T, distance_method))), 
                         index=cus_matrix.columns, columns=cus_matrix.columns)
ranked = get_ranked_by5(cus_id, distances, buy_each)
customer = prods_new_customer
ranked_not_buyed = get_ranked_not_buyed_by(cus_id, ranked, customer_products_search)
filtered = get_filtered_matrix(cus_id, cus_matrix, ranked_not_buyed)
dic = {}
dic[cus_id] = get_dic(filtered)
print(dic) 


prods_new_customer


,ProductName,Quantity,CustomerID
0,Cookies - Assorted,3,100001
1,Flavouring - Orange,3,100001
2,Fenngreek Seed,1,100001
3,Wine - White Cab Sauv.on,1,100001
4,Bandage - Flexible Neon,3,100001


*****  RECOMMENDATIONS  *****


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


{'100001': {'product1': 'Appetizer - Mini Egg Roll, Shrimp', 'product2': 'Appetizer - Mushroom Tart', 'product3': 'Appetizer - Sausage Rolls', 'product4': 'Apricots - Dried', 'product5': 'Baking Powder'}}


/anaconda3/lib/python3.6/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


### Step 8: Recommend 5 products to this new customer using a item similarity approach.

In [22]:
prods_new_customer = pd.DataFrame.from_dict(new_customer, orient='index').reset_index()
prods_new_customer.columns = ['ProductName', 'Quantity']
prods_new_customer['CustomerID'] = 100001
prods_new_customer.head()

print('*****  RECOMMENDATIONS BY ITEM SIMMILARITY  *****')
print(get_recommendations(100001, list_prods=get_unique_prods(prods_new_customer)))


,ProductName,Quantity,CustomerID
0,Cookies - Assorted,3,100001
1,Flavouring - Orange,3,100001
2,Fenngreek Seed,1,100001
3,Wine - White Cab Sauv.on,1,100001
4,Bandage - Flexible Neon,3,100001


*****  RECOMMENDATIONS BY ITEM SIMMILARITY  *****
['V8 - Berry Blend', 'Scallops - Live In Shell', 'Hersey Shakes', 'Beef - Chuck, Boneless', 'Chocolate - Semi Sweet, Calets']
